In [1]:
%run packages.py

/Users/mcbookairdebat/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from data_import_enginnering import DataImportEnginnering
from metric_selection import Pair_Selection
from data_visualization import Data_Visualization
import importlib

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# 1. DATA

## 1.1. Importation of data into a dictionary

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
Stock_list = pd.DataFrame(tables[0])
Stock_list.drop(['Security','GICS Sub-Industry','Headquarters Location','CIK','Founded'],axis = 1, inplace=True)
Stock_list['Date added'] = pd.to_datetime(Stock_list['Date added'])
Stock_list = Stock_list[( Stock_list['Date added'] < '2023-12-31') & (Stock_list['Date added']>'2015-01-01')][['Symbol','GICS Sector']]

In [4]:
stock_list = list(Stock_list['Symbol'].unique())
Stock_list_sector = Stock_list.groupby('GICS Sector')['Symbol'].unique().to_dict()
sector_list = list(Stock_list_sector.keys())
pairs = {k:[tuple(sorted((i,j))) for i in Stock_list_sector[k] for j in Stock_list_sector[k] if j!=i] for k in sector_list}
pairs = {k :sorted(list(set(pairs[k]))) for k in sector_list}

In [5]:
start_date = '2018-01-01'
end_date = '2019-12-31'
period = '1d'
colone = ['Close']


In [6]:
Enginner = DataImportEnginnering(stock_list,start_date=start_date,end_date=end_date,period=period,colone=colone)

In [7]:
raw_data = Enginner.RawData()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABNB']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2018-01-01 -> 2019-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1514782800, endDate = 1577768400")')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[******************

## 1.2. Normalisation of data

In [8]:
raw_data_normalized = {}
for key, value in raw_data.items():
    raw_data_normalized[key] = Enginner.normalize(value)

raw_data_normalized

{'AOS': Price          Close
 Ticker           AOS
 Date                
 2018-01-02  0.761193
 2018-01-03  0.775196
 2018-01-04  0.786171
 2018-01-05  0.816448
 2018-01-08  0.827802
 ...              ...
 2019-12-23  0.275176
 2019-12-24  0.289248
 2019-12-26  0.298629
 2019-12-27  0.301365
 2019-12-30  0.291984
 
 [502 rows x 1 columns],
 'AMD': Price          Close
 Ticker           AMD
 Date                
 2018-01-02  0.039084
 2018-01-03  0.054447
 2018-01-04  0.069811
 2018-01-05  0.063342
 2018-01-08  0.074124
 ...              ...
 2019-12-23  0.968464
 2019-12-24  0.997574
 2019-12-26  1.000000
 2019-12-27  0.987871
 2019-12-30  0.970081
 
 [502 rows x 1 columns],
 'ABNB': Empty DataFrame
 Columns: [(Close, ABNB)]
 Index: [],
 'ALB': Price          Close
 Ticker           ALB
 Date                
 2018-01-02  0.922864
 2018-01-03  0.926252
 2018-01-04  0.905796
 2018-01-05  0.925341
 2018-01-08  1.000000
 ...              ...
 2019-12-23  0.185756
 2019-12-24  0.184273
 201

In [9]:
dataframe_normalized = Enginner.Dictionnary_to_Dataframe(raw_data_normalized)
raw_dataframe = Enginner.Dictionnary_to_Dataframe(raw_data)


/Users/mcbookairdebat/Desktop/Projet/project/GitHub/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[stock_name] = z[stock_name]
/Users/mcbookairdebat/Desktop/Projet/project/GitHub/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[stock_name] = z[stock_name]
/Users/mcbookairdebat/Desktop/Projet/project/GitHub/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:40: PerformanceWarning: Da

In [10]:
dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()
raw_dataframe= raw_dataframe.stack().stack().droplevel(level =2).unstack()

/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_44529/628406389.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()
/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_44529/628406389.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  raw_dataframe= raw_dataframe.stack().stack().droplevel(level =2).unstack()


## 1.3 Correlation Matrix

In [11]:
q = raw_dataframe.shape[1]/raw_dataframe.shape[0]
data = raw_dataframe.copy()
data = np.log(data/data.shift(1))
data = data.apply(lambda x: x-x.mean())
data = data.iloc[1:,:]


In [12]:
correlation_matrix = data.corr()

# 2.Reduction Dimension with PCA and Marcenko Pastur 

## Marcenko pastur to keep only the rights one

In [13]:

def mpPDF(var,q):
    e_min,e_max = var* (1 -(1/q)**.5),var* (1 + (1/q)**.5)
    return e_min,e_max

from sklearn.neighbors._kde import KernelDensity
def getPCA(matrix):
    eVal,eVec = np.linalg.eigh(matrix)
    indices = eVal.argsort()[::-1]
    eVal,eVec = eVal[indices],eVec[:,indices]
    eVal = np.diagflat(eVal)
    return eVal,eVec

def fitKDE(obs,bWidth =.25,kernel = 'gaussian', x=None):
    if len(obs.shape)==1: obs =obs.reshape(-1,1)
    kde = KernelDensity(bandwidth=bWidth,kernel=kernel).fit(obs)
    if x is None : x=np.unique(obs).reshape(-1,1)
    if len(x.shape)==1 : x=x.reshape(-1,1)
    logProb = kde.score_samples(x)
    pdf = pd.Series(np.exp(logProb), index = x.flatten())
    return pdf

x = np.random.normal(size=(1000,1000))
eVal0,eVec0 = getPCA(correlation_matrix)
e_min,e_max = mpPDF(1.,raw_dataframe.shape[1]/raw_dataframe.shape[0])
mask = (np.diag(eVal0)>e_max)

In [14]:
mt =mask*np.diag(eVal0)
treshold = (mt>0).sum()
eVec0 = pd.DataFrame(eVec0).T


In [15]:
pca_proj = eVec0.iloc[:,:3]
new_corr = np.dot(correlation_matrix.T,pca_proj)
new_corr = pd.DataFrame(new_corr,index = correlation_matrix.index,columns =['X','Y','Z'])

# 3.Applying Optic/ DBScan and construction pair 

In [16]:
# Research 
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


X = StandardScaler().fit_transform(new_corr)
X =pd.DataFrame(X,index = new_corr.index, columns = ['X','Y','Z'])
X

,X,Y,Z
Ticker,,,
AOS,-0.426862,1.165061,0.612200
AMD,-1.446483,0.071669,0.282285
ALB,-0.977558,1.044532,-0.723239
ARE,0.936313,-2.207424,-0.996530
ALGN,-1.285193,-0.720796,0.236157
...,...,...,...
FOXA,0.604286,1.564717,-1.220368
FOX,0.547515,1.652856,-1.247049
DOW,-0.729702,2.551139,-0.862477


In [17]:
import numpy as np

from sklearn import metrics
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.3, min_samples=3).fit(X)
labels = db.labels_
X['label'] =labels

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 11
Estimated number of noise points: 91


In [18]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(X, x='X', y='Y', z='Y',color='label')
fig.show()

In [19]:
X.reset_index(inplace=True, drop = False)


In [20]:
f_dic ={}
for tick in X.label.unique():
    if tick ==-1:
        pass
    else:
        f_dic[tick] = X[X['label']==tick]['Ticker']

In [21]:
pairs=[]
for val in f_dic.values():
    val1 = val.copy()
    for stock1 in val1:
        pairs.extend([(stock1,stock2)  for stock2 in val1 if stock2!=stock1])
        val1 = val1[1:]

# 4.Rules for pair selection 

## Check for cointegration

In [24]:
# Engle granger 
# first cointegration step 
Metric = Pair_Selection(dataframe_normalized,number_of_pair=2,stock_list_sector=Stock_list_sector,sector_list=sector_list,sector_neutral = True)

In [25]:
selected_pair = Metric.paire_selection_2(pairs,p=0.05)

## Check that the spread is mean reverting

In [24]:
# Hurst Exponent 


## Check that the spread is tradable 

In [25]:
#

## Check spread reversion is consistent 


In [26]:
# mean_cross superior to 12 